In [2]:
import requests
from selenium import webdriver
from webdriver_manager.utils import chrome_version
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd
# これをimportしておくことで自分でcheromedriverをインストールする必要がない
import chromedriver_binary
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [3]:
cols1 = ["ユーザー", "ユーザーURL","記事タイトル","記事URL", "記事いいね数", "記事作成日"]
cols2 = ["ユーザーURL", "プロフィール", "総いいね数", "フォロワー", "github", "twitter", "ポートフォリオ", "Article数", "Scraps数"]

In [4]:
class GetZenn:
    def __init__(self):
        #seleniumのオプション
        self.options=Options()
        self.options.add_argument("no-sandbox")
        self.options.add_argument("--disable-extensions")
#         self.options.add_argument("--headless")
        self.options.add_argument('--disable-gpu')  
        self.options.add_argument('--ignore-certificate-errors')
        self.options.add_argument('--allow-running-insecure-content')
        self.options.add_argument('--disable-web-security')
        self.options.add_argument('--disable-desktop-notifications')
        self.options.add_argument("--disable-extensions")
        self.options.add_argument('--lang=ja')
        self.options.add_argument('--blink-settings=imagesEnabled=false')
        self.options.add_argument('--disable-dev-shm-usage')
        self.options.add_argument('--proxy-server="direct://"')
        self.options.add_argument('--proxy-bypass-list=*')    
        self.options.add_argument('--start-maximized')
        #twitterのログイン多面
        self.home_url="https://zenn.dev/articles?page=1"
    
    def first_driver(self):
        driver = webdriver.Chrome(chrome_options=self.options)
        driver.get(self.home_url)
        #urlの読み込みに時間がかかる可能性があるため
        time.sleep(5)
        return driver

    def from_article(self, driver):
        #初期化
        dates = []
        titles  = []
        urls = []
        usernames = []
        likes =  []
        pros = []
        
        #現在のページ数
        page_num =1
        judge =True
        
        
        while judge == True:
            print("{}ページ目を進行中".format(page_num))
            # 1ページあたり42記事
            kiji_containers = driver.find_elements_by_class_name("ArticleList_itemContainer__1ECL5")
            #①要素の取得
            try:
                for i in range(len(kiji_containers)):
                    #各種要素の取得
                    date = kiji_containers[i].find_element_by_tag_name("time").get_attribute("datetime")
                    
                    title = kiji_containers[i].find_element_by_class_name("ArticleList_title__3KqyY").text
                    
                    url = kiji_containers[i].find_element_by_class_name("ArticleList_emoji__1RiRB").get_attribute("href")
             
                    username = kiji_containers[i].find_element_by_class_name("ArticleList_userName__3MtlK").text

                    pro = kiji_containers[i].find_element_by_class_name("ArticleList_user__1XkRB").get_attribute("href")
                   
                    
                    #likeはない可能性もあるので
                    try:
                        like = kiji_containers[i].find_element_by_class_name("ArticleList_like__3OVME").text
                    except:
                        like =  "0"
                    
                    dates.append(date)
                    titles.append(title)
                    urls.append(url)
                    usernames.append(username)
                    likes.append(like)
                    pros.append(pro)
                    
                    
            except KeyboardInterrupt:
                judge=False
                
            except:
                print("要素を取得できませんでした。{}ページ目".format(page_num))
            
                
        #②pageのチェック
            try:
                # ページネーションの部分
                next_= driver.find_element_by_class_name("ListPagination_linkNext__2-ejg")
                #ページのチェック
                next_.click()
                time.sleep(1)
                page_num+=1
                
            except KeyboardInterrupt:
                judge=False

            except:
                judge = False
                print("もう記事がないので修了します")
                page_num+=1
            
        driver.quit()    
            
            
        return usernames, pros, titles, urls, likes, dates
    
    def auto_csv(self, datas:list, cols:list, file_name:str):

        if len(datas) != len(cols):
            return "希望のデータのカラムと作成したデータのカラムの数が異なります"

        else:
            pd_datas = []

            for data in datas:
                pd_datas.append(pd.DataFrame(data))

            concat = pd.concat([li.T for li in pd_datas]).T
            concat.columns = cols
            
            concat.to_csv("{}.csv".format(file_name))
            return concat    
        
        
    #df_usersは userのurlのシリーズ
    def profile_get(self, driver, df_users):
        users = []
        descriptions = []
        all_likes = []
        followers = []
        githubs = []
        twitters = []
        my_urls = []
        article_nums = []
        scrap_nums  = []

        user_num = 1

        try:
            for df_user in df_users:
                print("{}/{}を進行中".format(user_num, len(df_users)))

                #　ユーザーページヘジャンプ
                driver.get(df_user)
                time.sleep(0.5)

                # descriptionについて
                try:
                    description = driver.find_element_by_class_name("link-text-inside").text
                except:
                    description = "なし"

                # すべての記事のいいね数について
                try:
                    all_like = driver.find_element_by_xpath('//button[contains(text(), "Likes")]').text.replace("\nLikes", "")
                except:
                    all_like = "0"

                # フォロワーについて
                try:
                    follower = driver.find_element_by_xpath('//button[contains(text(), "Followers")]').text.replace("\nFollowers", "")
                except:
                    follower = "0"

                # githubについて
                try:
                    github = driver.find_element_by_xpath('//a[contains(@href, "github")]').get_attribute("href")
                except:
                    github = "none"

                # twitterについて
                try:
                    twitter = driver.find_element_by_xpath('//a[contains(@href, "twitter")]').get_attribute("href")
                except:
                    twitter = "none"

                # 自分のリンクについて
                try:
                    my_url = driver.find_element_by_class_name('tooltip-container').get_attribute("href")
                except:
                    my_url = "none"

                # articleの数
                try:
                    article_num = driver.find_element_by_xpath('//a[contains(text(), "Articles")]').text.replace("Articles", "")
                except:
                    article_num = "0"

                # scrapsの数
                try:
                    scrap_num = driver.find_element_by_xpath('//a[contains(text(), "Scraps")]').text.replace("Scraps", "")
                except:
                    scrap_num = "0"

                users.append(df_user)
                descriptions.append(description)
                all_likes.append(all_like)
                followers.append(follower)
                githubs.append(github)
                twitters.append(twitter)
                my_urls.append(my_url)
                article_nums.append(article_num)
                scrap_nums.append(scrap_num)

                user_num+=1

        except KeyboardInterrupt:
            print("{}ユーザー目で中断されました".format(user_num))

        except:
            print("{}ユーザーめで何らかのエラーが発生しました".format(user_num))

        data_list = [users, descriptions, all_likes, followers, githubs,  twitters, my_urls,  article_nums, scrap_nums]
        return data_list, users, descriptions, all_likes, followers, githubs,  twitters, my_urls,  article_nums, scrap_nums
        
        

In [5]:
def main1():
    zenn = GetZenn()
    driver = zenn.first_driver()
    usernames, pros, titles, urls, likes, dates= zenn.from_article(driver)
    data_list = [usernames, pros, titles, urls, likes, dates]
    return zenn, data_list, usernames, pros, titles, urls, likes, dates

In [6]:
def main2(df_users):
    zenn = GetZenn()
    driver = zenn.first_driver()
    data_list, users, descriptions, all_likes, followers, githubs,  twitters, my_urls,  article_nums, scrap_nums=zenn.profile_get(driver, df_users)
    return data_list

In [47]:
zenn, data_list, usernames, pros, titles, urls, likes, dates = main1()

/Users/ryusei/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:24: DeprecationWarning: use options instead of chrome_options


1ページ目を進行中
2ページ目を進行中
要素を取得できませんでした。2ページ目
もう記事がないので修了します


In [48]:
data_list

[['Takepepe',
  'pirosuke',
  'wok',
  'ほんまやで',
  'nakayasu-yuichi',
  'tanihiro',
  'komisan19',
  'うぽぽん',
  'kehra',
  'かるでね',
  'shiozumi',
  'ゆーたろー',
  '黒ヰ樹',
  'YAMADA Nobuko',
  'TAK',
  'kawakawaryuryu',
  'とが',
  'Kazuki Yonemoto',
  '雪猫',
  '水道屋さん',
  'jwnr',
  'mongolyy',
  'fuchami',
  'gon',
  'youta',
  'Ameta',
  'karamage',
  'Kyome',
  'shin_semiya',
  'Ayato N',
  'Godai Hori',
  'kaki',
  'しもしゃん',
  'ほげいほげみ',
  'barusu',
  'あおいくん',
  'なかつがわ',
  'あーる',
  'kiiimii',
  'BB',
  'fiord',
  'yuuyu',
  'kitasuke',
  'tajima_taso',
  'しゅん',
  'Yusuke',
  'enuenu',
  'kyoncy',
  'JAS',
  'mochiblock',
  'shamijohn123',
  'kj_trsm',
  'ougi',
  'catnose',
  'UXPin Official',
  'うえぞう',
  'つらら',
  'Kazuki Ota',
  '21f',
  'hashito',
  'ima',
  'かろてん',
  'nitaking',
  'niisan',
  'shiozumi',
  'wing3298',
  'かるでね',
  'koralle',
  'マインドウッド',
  'kazuhe',
  'slowhand',
  'Ryo Atsuta',
  'ymd',
  'Tomohiko Himura',
  'IKEH',
  '水道屋さん',
  'harasho',
  'peperoncicicino',
  'Kenji Furuno

In [49]:
zenn.auto_csv(data_list, cols1, "sample.csv")

,ユーザー,ユーザーURL,記事タイトル,記事URL,記事いいね数,記事作成日
0,Takepepe,https://zenn.dev/takepepe,React.Context で作る GlobalUI Custom Hooks,https://zenn.dev/takepepe/articles/context-cus...,9,2021-02-16T02:54:07+00:00
1,pirosuke,https://zenn.dev/pirosuke,VueベースなWebツールのロジック部分をRust製Wasmで実装する,https://zenn.dev/pirosuke/articles/vue-with-wasm,1,2021-02-16T02:49:53+00:00
2,wok,https://zenn.dev/wok,Amazon Chime SDKで音声からノイズを除去する,https://zenn.dev/wok/articles/0011_amaznon-chi...,1,2021-02-16T02:36:10+00:00
3,ほんまやで,https://zenn.dev/hommayade,【Laravel】SFTPをLaravelで使うために【備忘録】,https://zenn.dev/hommayade/articles/a626650b89...,2,2021-02-16T01:34:41+00:00
4,nakayasu-yuichi,https://zenn.dev/nkysyuichi,[iOSアプリ開発] ファイルを抽象化した構造体を作る(5),https://zenn.dev/nkysyuichi/articles/0689c909b...,1,2021-02-16T01:32:41+00:00
...,...,...,...,...,...,...
75,水道屋さん,https://zenn.dev/suiudou,dockerさんのコンテナAPIとkubernetesの実装形式,https://zenn.dev/suiudou/articles/64e1bd75f0bb97,0,2021-02-15T00:49:25+09:00
76,harasho,https://zenn.dev/harasho,Vuetifyのv-checkboxで最低一つ選択必須にするバリデーションの実装,https://zenn.dev/harasho/articles/validate-che...,2,2021-02-15T00:43:01+09:00
77,peperoncicicino,https://zenn.dev/peperoncicicino,遊戯王カードをドローする GitHub Action を作りました,https://zenn.dev/peperoncicicino/articles/bbb4...,26,2021-02-15T00:23:23+09:00
78,Kenji Furuno,https://zenn.dev/pantone170145,AWS AmplifyでSSOアカウントを利用する,https://zenn.dev/pantone170145/articles/aws-am...,2,2021-02-15T00:12:12+09:00


In [246]:
concat  = zenn.auto_csv(datas, cols1 ,"user情報")

In [7]:
df1 = pd.read_csv("userinfo_from_article.csv")["ユーザーURL"][:1500]
df2 = pd.read_csv("userinfo_from_article.csv")["ユーザーURL"][1500:3000]
df3 = pd.read_csv("userinfo_from_article.csv")["ユーザーURL"][3000:4500]
df4 = pd.read_csv("userinfo_from_article.csv")["ユーザーURL"][4500:]

In [8]:
data_list = main2(df1)
zenn = GetZenn()
con = zenn.auto_csv(data_list, cols2, "start_1500")

/Users/ryusei/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:24: DeprecationWarning: use options instead of chrome_options


1/1500を進行中
2/1500を進行中
3/1500を進行中
4/1500を進行中
5/1500を進行中
5ユーザーめで何らかのエラーが発生しました


In [9]:
con

,ユーザーURL,プロフィール,総いいね数,フォロワー,github,twitter,ポートフォリオ,Article数,Scraps数
0,https://zenn.dev/shimotaroo,2020年4月から関西でWebエンジニア。PHP/Laravel/jQuery/Vue.js...,35,3,https://github.com/shimotaroo,https://twitter.com/shimotaroo,https://linktr.ee/shimotaroo,14,0
1,https://zenn.dev/barusu,趣味で馬をやっている者だ,88,13,https://github.com/zenn-dev/zenn-roadmap/proje...,https://twitter.com/zenn_dev,none,43,2
2,https://zenn.dev/j_09,はじめまして！ プログラミング初学者です！ よろしくおねがいします！,37,2,https://github.com/junono09,https://twitter.com/jo51608309,none,27,3
3,https://zenn.dev/centerfield77,なし,0,0,none,none,none,0,0


In [277]:
data_list2 = main2(df2)
zenn = GetZenn()
con2 = zenn.auto_csv(data_list2, cols2, "1500_3000")

/Users/ryusei/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:24: DeprecationWarning: use options instead of chrome_options


1/1500を進行中
2/1500を進行中
3/1500を進行中
4/1500を進行中
5/1500を進行中
6/1500を進行中
7/1500を進行中
8/1500を進行中
9/1500を進行中
10/1500を進行中
11/1500を進行中
12/1500を進行中
13/1500を進行中
14/1500を進行中
15/1500を進行中
16/1500を進行中
17/1500を進行中
18/1500を進行中
19/1500を進行中
20/1500を進行中
21/1500を進行中
22/1500を進行中
23/1500を進行中
24/1500を進行中
25/1500を進行中
26/1500を進行中
27/1500を進行中
28/1500を進行中
29/1500を進行中
30/1500を進行中
31/1500を進行中
32/1500を進行中
33/1500を進行中
34/1500を進行中
35/1500を進行中
36/1500を進行中
37/1500を進行中
38/1500を進行中
39/1500を進行中
40/1500を進行中
41/1500を進行中
42/1500を進行中
43/1500を進行中
44/1500を進行中
45/1500を進行中
46/1500を進行中
47/1500を進行中
48/1500を進行中
49/1500を進行中
50/1500を進行中
51/1500を進行中
52/1500を進行中
53/1500を進行中
54/1500を進行中
55/1500を進行中
56/1500を進行中
57/1500を進行中
58/1500を進行中
59/1500を進行中
60/1500を進行中
61/1500を進行中
62/1500を進行中
63/1500を進行中
64/1500を進行中
65/1500を進行中
66/1500を進行中
67/1500を進行中
68/1500を進行中
69/1500を進行中
70/1500を進行中
71/1500を進行中
72/1500を進行中
73/1500を進行中
74/1500を進行中
75/1500を進行中
76/1500を進行中
77/1500を進行中
78/1500を進行中
79/1500を進行中
80/1500を進行中
81/1500を進行中
82/1500を進行中
83/1500を進行中
84/1500を進行中
8

640/1500を進行中
641/1500を進行中
642/1500を進行中
643/1500を進行中
644/1500を進行中
645/1500を進行中
646/1500を進行中
647/1500を進行中
648/1500を進行中
649/1500を進行中
650/1500を進行中
651/1500を進行中
652/1500を進行中
653/1500を進行中
654/1500を進行中
655/1500を進行中
656/1500を進行中
657/1500を進行中
658/1500を進行中
659/1500を進行中
660/1500を進行中
661/1500を進行中
662/1500を進行中
663/1500を進行中
664/1500を進行中
665/1500を進行中
666/1500を進行中
667/1500を進行中
668/1500を進行中
669/1500を進行中
670/1500を進行中
671/1500を進行中
672/1500を進行中
673/1500を進行中
674/1500を進行中
675/1500を進行中
676/1500を進行中
677/1500を進行中
678/1500を進行中
679/1500を進行中
680/1500を進行中
681/1500を進行中
682/1500を進行中
683/1500を進行中
684/1500を進行中
685/1500を進行中
686/1500を進行中
687/1500を進行中
688/1500を進行中
689/1500を進行中
690/1500を進行中
691/1500を進行中
692/1500を進行中
693/1500を進行中
694/1500を進行中
695/1500を進行中
696/1500を進行中
697/1500を進行中
698/1500を進行中
699/1500を進行中
700/1500を進行中
701/1500を進行中
702/1500を進行中
703/1500を進行中
704/1500を進行中
705/1500を進行中
706/1500を進行中
707/1500を進行中
708/1500を進行中
709/1500を進行中
710/1500を進行中
711/1500を進行中
712/1500を進行中
713/1500を進行中
714/1500を進行中
715/1500を進行中
716/1500を進行中

1251/1500を進行中
1252/1500を進行中
1253/1500を進行中
1254/1500を進行中
1255/1500を進行中
1256/1500を進行中
1257/1500を進行中
1258/1500を進行中
1259/1500を進行中
1260/1500を進行中
1261/1500を進行中
1262/1500を進行中
1263/1500を進行中
1264/1500を進行中
1265/1500を進行中
1266/1500を進行中
1267/1500を進行中
1268/1500を進行中
1269/1500を進行中
1270/1500を進行中
1271/1500を進行中
1272/1500を進行中
1273/1500を進行中
1274/1500を進行中
1275/1500を進行中
1276/1500を進行中
1277/1500を進行中
1278/1500を進行中
1279/1500を進行中
1280/1500を進行中
1281/1500を進行中
1282/1500を進行中
1283/1500を進行中
1284/1500を進行中
1285/1500を進行中
1286/1500を進行中
1287/1500を進行中
1288/1500を進行中
1289/1500を進行中
1290/1500を進行中
1291/1500を進行中
1292/1500を進行中
1293/1500を進行中
1294/1500を進行中
1295/1500を進行中
1296/1500を進行中
1297/1500を進行中
1298/1500を進行中
1299/1500を進行中
1300/1500を進行中
1301/1500を進行中
1302/1500を進行中
1303/1500を進行中
1304/1500を進行中
1305/1500を進行中
1306/1500を進行中
1307/1500を進行中
1308/1500を進行中
1309/1500を進行中
1310/1500を進行中
1311/1500を進行中
1312/1500を進行中
1313/1500を進行中
1314/1500を進行中
1315/1500を進行中
1316/1500を進行中
1317/1500を進行中
1318/1500を進行中
1319/1500を進行中
1320/1500を進行中
1321/1500を進行中
1322/1

In [279]:
data_list3 = main2(df3)
zenn = GetZenn()
con3 = zenn.auto_csv(data_list3, cols2, "3000_4500")

/Users/ryusei/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:24: DeprecationWarning: use options instead of chrome_options


1/1500を進行中
2/1500を進行中
3/1500を進行中
4/1500を進行中
5/1500を進行中
6/1500を進行中
7/1500を進行中
8/1500を進行中
9/1500を進行中
10/1500を進行中
11/1500を進行中
12/1500を進行中
13/1500を進行中
14/1500を進行中
15/1500を進行中
16/1500を進行中
17/1500を進行中
18/1500を進行中
19/1500を進行中
20/1500を進行中
21/1500を進行中
22/1500を進行中
23/1500を進行中
24/1500を進行中
25/1500を進行中
26/1500を進行中
27/1500を進行中
28/1500を進行中
29/1500を進行中
30/1500を進行中
31/1500を進行中
32/1500を進行中
33/1500を進行中
34/1500を進行中
35/1500を進行中
36/1500を進行中
37/1500を進行中
38/1500を進行中
39/1500を進行中
40/1500を進行中
41/1500を進行中
42/1500を進行中
43/1500を進行中
44/1500を進行中
45/1500を進行中
46/1500を進行中
47/1500を進行中
48/1500を進行中
49/1500を進行中
50/1500を進行中
51/1500を進行中
52/1500を進行中
53/1500を進行中
54/1500を進行中
55/1500を進行中
56/1500を進行中
57/1500を進行中
58/1500を進行中
59/1500を進行中
60/1500を進行中
61/1500を進行中
62/1500を進行中
63/1500を進行中
64/1500を進行中
65/1500を進行中
66/1500を進行中
67/1500を進行中
68/1500を進行中
69/1500を進行中
70/1500を進行中
71/1500を進行中
72/1500を進行中
73/1500を進行中
74/1500を進行中
75/1500を進行中
76/1500を進行中
77/1500を進行中
78/1500を進行中
79/1500を進行中
80/1500を進行中
81/1500を進行中
82/1500を進行中
83/1500を進行中
84/1500を進行中
8

640/1500を進行中
641/1500を進行中
642/1500を進行中
643/1500を進行中
644/1500を進行中
645/1500を進行中
646/1500を進行中
647/1500を進行中
648/1500を進行中
649/1500を進行中
650/1500を進行中
651/1500を進行中
652/1500を進行中
653/1500を進行中
654/1500を進行中
655/1500を進行中
656/1500を進行中
657/1500を進行中
658/1500を進行中
659/1500を進行中
660/1500を進行中
661/1500を進行中
662/1500を進行中
663/1500を進行中
664/1500を進行中
665/1500を進行中
666/1500を進行中
667/1500を進行中
668/1500を進行中
669/1500を進行中
670/1500を進行中
671/1500を進行中
672/1500を進行中
673/1500を進行中
674/1500を進行中
675/1500を進行中
676/1500を進行中
677/1500を進行中
678/1500を進行中
679/1500を進行中
680/1500を進行中
681/1500を進行中
682/1500を進行中
683/1500を進行中
684/1500を進行中
685/1500を進行中
686/1500を進行中
687/1500を進行中
688/1500を進行中
689/1500を進行中
690/1500を進行中
691/1500を進行中
692/1500を進行中
693/1500を進行中
694/1500を進行中
695/1500を進行中
696/1500を進行中
697/1500を進行中
698/1500を進行中
699/1500を進行中
700/1500を進行中
701/1500を進行中
702/1500を進行中
703/1500を進行中
704/1500を進行中
705/1500を進行中
706/1500を進行中
707/1500を進行中
708/1500を進行中
709/1500を進行中
710/1500を進行中
711/1500を進行中
712/1500を進行中
713/1500を進行中
714/1500を進行中
715/1500を進行中
716/1500を進行中

1251/1500を進行中
1252/1500を進行中
1253/1500を進行中
1254/1500を進行中
1255/1500を進行中
1256/1500を進行中
1257/1500を進行中
1258/1500を進行中
1259/1500を進行中
1260/1500を進行中
1261/1500を進行中
1262/1500を進行中
1263/1500を進行中
1264/1500を進行中
1265/1500を進行中
1266/1500を進行中
1267/1500を進行中
1268/1500を進行中
1269/1500を進行中
1270/1500を進行中
1271/1500を進行中
1272/1500を進行中
1273/1500を進行中
1274/1500を進行中
1275/1500を進行中
1276/1500を進行中
1277/1500を進行中
1278/1500を進行中
1279/1500を進行中
1280/1500を進行中
1281/1500を進行中
1282/1500を進行中
1283/1500を進行中
1284/1500を進行中
1285/1500を進行中
1286/1500を進行中
1287/1500を進行中
1288/1500を進行中
1289/1500を進行中
1290/1500を進行中
1291/1500を進行中
1292/1500を進行中
1293/1500を進行中
1294/1500を進行中
1295/1500を進行中
1296/1500を進行中
1297/1500を進行中
1298/1500を進行中
1299/1500を進行中
1300/1500を進行中
1301/1500を進行中
1302/1500を進行中
1303/1500を進行中
1304/1500を進行中
1305/1500を進行中
1306/1500を進行中
1307/1500を進行中
1308/1500を進行中
1309/1500を進行中
1310/1500を進行中
1311/1500を進行中
1312/1500を進行中
1313/1500を進行中
1314/1500を進行中
1315/1500を進行中
1316/1500を進行中
1317/1500を進行中
1318/1500を進行中
1319/1500を進行中
1320/1500を進行中
1321/1500を進行中
1322/1

In [280]:
data_list4 = main2(df4)
zenn = GetZenn()
con4 = zenn.auto_csv(data_list4, cols2, "4500_last")

/Users/ryusei/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:24: DeprecationWarning: use options instead of chrome_options


1/2451を進行中
2/2451を進行中
3/2451を進行中
4/2451を進行中
5/2451を進行中
6/2451を進行中
7/2451を進行中
8/2451を進行中
9/2451を進行中
10/2451を進行中
11/2451を進行中
12/2451を進行中
13/2451を進行中
14/2451を進行中
15/2451を進行中
16/2451を進行中
17/2451を進行中
18/2451を進行中
19/2451を進行中
20/2451を進行中
21/2451を進行中
22/2451を進行中
23/2451を進行中
24/2451を進行中
25/2451を進行中
26/2451を進行中
27/2451を進行中
28/2451を進行中
29/2451を進行中
30/2451を進行中
31/2451を進行中
32/2451を進行中
33/2451を進行中
34/2451を進行中
35/2451を進行中
36/2451を進行中
37/2451を進行中
38/2451を進行中
39/2451を進行中
40/2451を進行中
41/2451を進行中
42/2451を進行中
43/2451を進行中
44/2451を進行中
45/2451を進行中
46/2451を進行中
47/2451を進行中
48/2451を進行中
49/2451を進行中
50/2451を進行中
51/2451を進行中
52/2451を進行中
53/2451を進行中
54/2451を進行中
55/2451を進行中
56/2451を進行中
57/2451を進行中
58/2451を進行中
59/2451を進行中
60/2451を進行中
61/2451を進行中
62/2451を進行中
63/2451を進行中
64/2451を進行中
65/2451を進行中
66/2451を進行中
67/2451を進行中
68/2451を進行中
69/2451を進行中
70/2451を進行中
71/2451を進行中
72/2451を進行中
73/2451を進行中
74/2451を進行中
75/2451を進行中
76/2451を進行中
77/2451を進行中
78/2451を進行中
79/2451を進行中
80/2451を進行中
81/2451を進行中
82/2451を進行中
83/2451を進行中
84/2451を進行中
8

640/2451を進行中
641/2451を進行中
642/2451を進行中
643/2451を進行中
644/2451を進行中
645/2451を進行中
646/2451を進行中
647/2451を進行中
648/2451を進行中
649/2451を進行中
650/2451を進行中
651/2451を進行中
652/2451を進行中
653/2451を進行中
654/2451を進行中
655/2451を進行中
656/2451を進行中
657/2451を進行中
658/2451を進行中
659/2451を進行中
660/2451を進行中
661/2451を進行中
662/2451を進行中
663/2451を進行中
664/2451を進行中
665/2451を進行中
666/2451を進行中
667/2451を進行中
668/2451を進行中
669/2451を進行中
670/2451を進行中
671/2451を進行中
672/2451を進行中
673/2451を進行中
674/2451を進行中
675/2451を進行中
676/2451を進行中
677/2451を進行中
678/2451を進行中
679/2451を進行中
680/2451を進行中
681/2451を進行中
682/2451を進行中
683/2451を進行中
684/2451を進行中
685/2451を進行中
686/2451を進行中
687/2451を進行中
688/2451を進行中
689/2451を進行中
690/2451を進行中
691/2451を進行中
692/2451を進行中
693/2451を進行中
694/2451を進行中
695/2451を進行中
696/2451を進行中
697/2451を進行中
698/2451を進行中
699/2451を進行中
700/2451を進行中
701/2451を進行中
702/2451を進行中
703/2451を進行中
704/2451を進行中
705/2451を進行中
706/2451を進行中
707/2451を進行中
708/2451を進行中
709/2451を進行中
710/2451を進行中
711/2451を進行中
712/2451を進行中
713/2451を進行中
714/2451を進行中
715/2451を進行中
716/2451を進行中

1251/2451を進行中
1252/2451を進行中
1253/2451を進行中
1254/2451を進行中
1255/2451を進行中
1256/2451を進行中
1257/2451を進行中
1258/2451を進行中
1259/2451を進行中
1260/2451を進行中
1261/2451を進行中
1262/2451を進行中
1263/2451を進行中
1264/2451を進行中
1265/2451を進行中
1266/2451を進行中
1267/2451を進行中
1268/2451を進行中
1269/2451を進行中
1270/2451を進行中
1271/2451を進行中
1272/2451を進行中
1273/2451を進行中
1274/2451を進行中
1275/2451を進行中
1276/2451を進行中
1277/2451を進行中
1278/2451を進行中
1279/2451を進行中
1280/2451を進行中
1281/2451を進行中
1282/2451を進行中
1283/2451を進行中
1284/2451を進行中
1285/2451を進行中
1286/2451を進行中
1287/2451を進行中
1288/2451を進行中
1289/2451を進行中
1290/2451を進行中
1291/2451を進行中
1292/2451を進行中
1293/2451を進行中
1294/2451を進行中
1295/2451を進行中
1296/2451を進行中
1297/2451を進行中
1298/2451を進行中
1299/2451を進行中
1300/2451を進行中
1301/2451を進行中
1302/2451を進行中
1303/2451を進行中
1304/2451を進行中
1305/2451を進行中
1306/2451を進行中
1307/2451を進行中
1308/2451を進行中
1309/2451を進行中
1310/2451を進行中
1311/2451を進行中
1312/2451を進行中
1313/2451を進行中
1314/2451を進行中
1315/2451を進行中
1316/2451を進行中
1317/2451を進行中
1318/2451を進行中
1319/2451を進行中
1320/2451を進行中
1321/2451を進行中
1322/2

1837/2451を進行中
1838/2451を進行中
1839/2451を進行中
1840/2451を進行中
1841/2451を進行中
1842/2451を進行中
1843/2451を進行中
1844/2451を進行中
1845/2451を進行中
1846/2451を進行中
1847/2451を進行中
1848/2451を進行中
1849/2451を進行中
1850/2451を進行中
1851/2451を進行中
1852/2451を進行中
1853/2451を進行中
1854/2451を進行中
1855/2451を進行中
1856/2451を進行中
1857/2451を進行中
1858/2451を進行中
1859/2451を進行中
1860/2451を進行中
1861/2451を進行中
1862/2451を進行中
1863/2451を進行中
1864/2451を進行中
1865/2451を進行中
1866/2451を進行中
1867/2451を進行中
1868/2451を進行中
1869/2451を進行中
1870/2451を進行中
1871/2451を進行中
1872/2451を進行中
1873/2451を進行中
1874/2451を進行中
1875/2451を進行中
1876/2451を進行中
1877/2451を進行中
1878/2451を進行中
1879/2451を進行中
1880/2451を進行中
1881/2451を進行中
1882/2451を進行中
1883/2451を進行中
1884/2451を進行中
1885/2451を進行中
1886/2451を進行中
1887/2451を進行中
1888/2451を進行中
1889/2451を進行中
1890/2451を進行中
1891/2451を進行中
1892/2451を進行中
1893/2451を進行中
1894/2451を進行中
1895/2451を進行中
1896/2451を進行中
1897/2451を進行中
1898/2451を進行中
1899/2451を進行中
1900/2451を進行中
1901/2451を進行中
1902/2451を進行中
1903/2451を進行中
1904/2451を進行中
1905/2451を進行中
1906/2451を進行中
1907/2451を進行中
1908/2

2423/2451を進行中
2424/2451を進行中
2425/2451を進行中
2426/2451を進行中
2427/2451を進行中
2428/2451を進行中
2429/2451を進行中
2430/2451を進行中
2431/2451を進行中
2432/2451を進行中
2433/2451を進行中
2434/2451を進行中
2435/2451を進行中
2436/2451を進行中
2437/2451を進行中
2438/2451を進行中
2439/2451を進行中
2440/2451を進行中
2441/2451を進行中
2442/2451を進行中
2443/2451を進行中
2444/2451を進行中
2445/2451を進行中
2446/2451を進行中
2447/2451を進行中
2448/2451を進行中
2449/2451を進行中
2450/2451を進行中
2451/2451を進行中


In [281]:
con4

,ユーザーURL,プロフィール,総いいね数,フォロワー,github,twitter,ポートフォリオ,Article数,Scraps数
0,https://zenn.dev/dzeyelid,Microsoft MVP (Azure). A Cloud developer who l...,107,15,https://github.com/dzeyelid,https://twitter.com/dz_,https://medium.com/@dzeyelid,11,4
1,https://zenn.dev/shotakaha,素粒子物理学実験してました。とある研究機構の科学広報（仮）です。合気道、自転車、カメラ、登山...,16,1,https://github.com/shotakaha,https://twitter.com/shotakaha,https://research.kek.jp/people/shotakah/,4,2
2,https://zenn.dev/matsuda,iOS Engineer,7,0,https://github.com/matsuda,https://twitter.com/mtsd,none,3,0
3,https://zenn.dev/tsubasa28,ベンチャー企業のWebエンジニアです！ 使用技術：Ruby/Rails/React/Redu...,15,0,https://github.com/zenn-dev/zenn-roadmap/proje...,https://twitter.com/zenn_dev,none,3,0
4,https://zenn.dev/kyome,火星に住む犬です。１日１Zenn。,84,13,https://github.com/kyome22,https://twitter.com/kyomesuke,https://kyome.io/index.html,18,4
...,...,...,...,...,...,...,...,...,...
2446,https://zenn.dev/suztomo,人を憶えるメモリオというアプリを作っています apps.apple.com/app/appl...,2,0,https://github.com/suztomo,https://twitter.com/suztomo,none,1,0
2447,https://zenn.dev/kitar,hello,0,0,https://github.com/zenn-dev/zenn-roadmap/proje...,https://twitter.com/zenn_dev,none,1,0
2448,https://zenn.dev/ryo_kawamata,Frontend engineer / Misoca / TypeScript / Vue....,865,98,https://github.com/kawamataryo,https://twitter.com/kawamataryo,https://qiita.com/ryo2132,31,1
2449,https://zenn.dev/shogo,Retoryというチャットアプリを運営しています。 FlutterとFirebaseが得意です,22,0,https://github.com/yshogo,https://twitter.com/yshogo87,https://note.com/shogoyamada,2,0


In [288]:
concon = pd.concat([con, con2, con3, con4]).reset_index(drop=True)

In [289]:
concon

,ユーザーURL,プロフィール,総いいね数,フォロワー,github,twitter,ポートフォリオ,Article数,Scraps数
0,https://zenn.dev/shimotaroo,2020年4月から関西でWebエンジニア。PHP/Laravel/jQuery/Vue.js...,24,1,https://github.com/shimotaroo,https://twitter.com/shimotaroo,https://linktr.ee/shimotaroo,11,0
1,https://zenn.dev/barusu,趣味で馬をやっている者だ,72,12,https://github.com/zenn-dev/zenn-roadmap/proje...,https://twitter.com/zenn_dev,none,34,2
2,https://zenn.dev/j_09,はじめまして！ プログラミング初学者です！ よろしくおねがいします！,35,2,https://github.com/junono09,https://twitter.com/jo51608309,none,24,3
3,https://zenn.dev/centerfield77,"1988年生まれ 東京在住 React, Djangoが好きです。 Qiita：qiita....",0,0,https://github.com/zenn-dev/zenn-roadmap/proje...,https://twitter.com/zenn_dev,none,1,0
4,https://zenn.dev/ikuya,Ubuntu Japanese Teamの人。VirtualBox翻訳者。team zpn主宰。,36,11,https://github.com/ikunya,https://twitter.com/ikunya,https://blog.goo.ne.jp/ikunya/,7,0
...,...,...,...,...,...,...,...,...,...
6946,https://zenn.dev/suztomo,人を憶えるメモリオというアプリを作っています apps.apple.com/app/appl...,2,0,https://github.com/suztomo,https://twitter.com/suztomo,none,1,0
6947,https://zenn.dev/kitar,hello,0,0,https://github.com/zenn-dev/zenn-roadmap/proje...,https://twitter.com/zenn_dev,none,1,0
6948,https://zenn.dev/ryo_kawamata,Frontend engineer / Misoca / TypeScript / Vue....,865,98,https://github.com/kawamataryo,https://twitter.com/kawamataryo,https://qiita.com/ryo2132,31,1
6949,https://zenn.dev/shogo,Retoryというチャットアプリを運営しています。 FlutterとFirebaseが得意です,22,0,https://github.com/yshogo,https://twitter.com/yshogo87,https://note.com/shogoyamada,2,0


In [324]:
ddf1  = pd.read_csv("zenn_info_from_pro.csv").iloc[:,1:]
ddf2 = pd.read_csv("userinfo_from_article.csv").iloc[:,1:]
ddf3 = pd.read_csv("zenn_twitter.csv").iloc[:,1:]

In [349]:
ddd1 = pd.read_csv("ZennリストALL.csv").reset_index(drop=True)

In [364]:
ddf3.twitter_id[3092]

'https://twitter.com/kotokaze__r'

In [356]:
no = 0
for i in range(len(ddd1)):
    if ddd1.Twitter[i] != ddf3.twitter_id[i]:
        no = i
print(no)

3092


In [333]:
ddf1.to_csv("zenn_profile改.csv")

In [338]:
ddf1["フォロワー"][3094]

0